In [1]:
import json
import os
import string
import pandas as pd
import plotly.graph_objects as go

pd.set_option('max_columns', None)
pd.set_option('max_row', None)

In [2]:
def loadmsgs(fn):
    if os.path.getsize(fn) <= 0:
        return [], []
    
    lstbg = []
    lstfg = []
    
    with open(fn, "r") as read_file:
        msgs = json.load(read_file)
        curfg = []
        isfg = False
        
        for v in msgs:
            co = json.loads(v)
            
            if 'allowedGameFeatures' not in co and 'error' not in co and 'game' in co:
                if co['game']['mode'] == 'NORMAL' and co['game']['nextMode'] == 'NORMAL':
                    lstbg.append(co)
                
                if co['game']['mode'] == 'NORMAL' and co['game']['nextMode'] == 'FREESPIN':
                    curfg = []
                    curfg.append(co)
                    isfg = True
                
                if isfg and co['game']['mode'] == 'FREESPIN':
                    curfg.append(co)
                    if co['game']['nextMode'] == 'NORMAL':
                        isfg = False
                        lstfg.append(curfg)
                
    
    return lstbg, lstfg

def loadAllFiles():
    lstbg = []
    lstfg = []
    
    dirs = os.listdir('./')
    for v in dirs:
        if v.endswith('.json'):
            clstbg, clstfg = loadmsgs(v)
            lstbg = lstbg + clstbg
            lstfg = lstfg + clstfg
    
    return lstbg, lstfg

def analyzeWins(lstbg, lstfg):
    wins = {}
    
    for v in lstbg:
        if 'realityCheck' in v:
            k = v['realityCheck']['winnings']
            if k in wins:
                wins[k] = wins[k] + 1
            else:
                wins[k] = 1
                
    for arr in lstfg:
        curwin = 0
        
        for v in arr:
            if 'realityCheck' in v:
                curwin = curwin + v['realityCheck']['winnings']
            
        k = curwin
        if k in wins:
            wins[k] = wins[k] + 1
        else:
            wins[k] = 1
    
    return wins

def analyzeReels(lstbg, lstfg, reelnum):
    bgreels = []
    fgreels = []    
    for i in range(reelnum):
        bgreels.append({})
        fgreels.append({})        
    
    for v in lstbg:
        vg = v['game']
        if 'reels' in vg:
            for i in range(len(vg['reels'])):
                reel = vg['reels'][i]

                for s in reel:
                    if s in bgreels[i]:
                        bgreels[i][s] = bgreels[i][s] + 1
                    else:
                        bgreels[i][s] = 1
                            
    for arr in lstfg:
        for i in range(len(arr)):
            vg = arr[i]['game']            
            
            if i == 0:
                if 'reels' in vg:
                    for i in range(len(vg['reels'])):
                        reel = vg['reels'][i]

                        for s in reel:
                            if s in bgreels[i]:
                                bgreels[i][s] = bgreels[i][s] + 1
                            else:
                                bgreels[i][s] = 1
            else:
                if 'reels' in vg:
                    for i in range(len(vg['reels'])):
                        reel = vg['reels'][i]

                        for s in reel:
                            if s in fgreels[i]:
                                fgreels[i][s] = fgreels[i][s] + 1
                            else:
                                fgreels[i][s] = 1                                         
    
    return bgreels, fgreels

def analyzeSymbolWins(lstbg, lstfg):
    bg = {}
    fg = {}
    
    for v in lstbg:
        vg = v['game']
        if 'betWayWins' in vg:
            for win in vg['betWayWins']:
                if win['symbol'] in bg:
                    if win['length'] in bg[win['symbol']]:
                        bg[win['symbol']][win['length']] = bg[win['symbol']][win['length']] + win['numberOfCombinations']
                    else:
                        bg[win['symbol']][win['length']] = win['numberOfCombinations']
                else:
                    bg[win['symbol']] = {}
                    bg[win['symbol']][win['length']] = win['numberOfCombinations']
                        
    for arr in lstfg:
        for i in range(len(arr)):
            vg = arr[i]['game']
            
            if i == 0:
                if 'betWayWins' in vg:
                    for win in vg['betWayWins']:
                        if win['symbol'] in bg:
                            if win['length'] in bg[win['symbol']]:
                                bg[win['symbol']][win['length']] = bg[win['symbol']][win['length']] + win['numberOfCombinations']
                            else:
                                bg[win['symbol']][win['length']] = win['numberOfCombinations']
                        else:
                            bg[win['symbol']] = {}
                            bg[win['symbol']][win['length']] = win['numberOfCombinations']
            else:
                if 'betWayWins' in vg:
                    for win in vg['betWayWins']:
                        if win['symbol'] in fg:
                            if win['length'] in fg[win['symbol']]:
                                fg[win['symbol']][win['length']] = fg[win['symbol']][win['length']] + win['numberOfCombinations']
                            else:
                                fg[win['symbol']][win['length']] = win['numberOfCombinations']
                        else:
                            fg[win['symbol']] = {}
                            fg[win['symbol']][win['length']] = win['numberOfCombinations']                
                        
    return bg, fg

def getAllReels(lstbg, lstfg):
    lstreels = {}
    
    for v in lstbg:
        if v['game']['reelsNextSpin'] in lstreels:
            lstreels[v['game']['reelsNextSpin']] = lstreels[v['game']['reelsNextSpin']] + 1
        else:
            lstreels[v['game']['reelsNextSpin']] = 1
            
    for arr in lstfg:
        for v in arr:
            if v['game']['reelsNextSpin'] in lstreels:
                lstreels[v['game']['reelsNextSpin']] = lstreels[v['game']['reelsNextSpin']] + 1
            else:
                lstreels[v['game']['reelsNextSpin']] = 1            
    
    return lstreels

def getAllModes(lstbg, lstfg):
    lstmodes = {}
    
    for v in lstbg:
        if v['game']['mode'] in lstmodes:
            lstmodes[v['game']['mode']] = lstmodes[v['game']['mode']] + 1
        else:
            lstmodes[v['game']['mode']] = 1
            
        if v['game']['nextMode'] in lstmodes:
            lstmodes[v['game']['nextMode']] = lstmodes[v['game']['nextMode']] + 1
        else:
            lstmodes[v['game']['nextMode']] = 1
            
    for arr in lstfg:
        for v in arr:
            if v['game']['mode'] in lstmodes:
                lstmodes[v['game']['mode']] = lstmodes[v['game']['mode']] + 1
            else:
                lstmodes[v['game']['mode']] = 1

            if v['game']['nextMode'] in lstmodes:
                lstmodes[v['game']['nextMode']] = lstmodes[v['game']['nextMode']] + 1
            else:
                lstmodes[v['game']['nextMode']] = 1            
    
    return lstmodes

def findMysterySymbolReveal(lstbg, lstfg):
    bg = {}
    fg = {}
    
    for v in lstbg:
        if v['game']['mysterySymbolReveal'] in bg:
            bg[v['game']['mysterySymbolReveal']] = bg[v['game']['mysterySymbolReveal']] + 1
        else:
            bg[v['game']['mysterySymbolReveal']] = 1
            
    for arr in lstfg:
        for i in range(len(arr)):
            v = arr[i]
            
            if i == 0:
                if v['game']['mysterySymbolReveal'] in bg:
                    bg[v['game']['mysterySymbolReveal']] = bg[v['game']['mysterySymbolReveal']] + 1
                else:
                    bg[v['game']['mysterySymbolReveal']] = 1            
            else:
                if v['game']['mysterySymbolReveal'] in fg:
                    fg[v['game']['mysterySymbolReveal']] = fg[v['game']['mysterySymbolReveal']] + 1
                else:
                    fg[v['game']['mysterySymbolReveal']] = 1                            
    
    return bg, fg

def findMystery(lst):
    lstm = []
    
    for v in lst:
        if v['game']['mysterySymbolReveal'] != '':
            lstm.append(v)
    
    return lstm

def genDataframeModeReels(lstbg, lstfg):
    modes = getAllModes(lstbg, lstfg)
    reels = getAllReels(lstbg, lstfg)
    totalspin = len(lstbg) + len(lstfg)

    data = {'mode':[], 'reels': [], 'spins': [], 'probability': [], 'winnings': [], 'rtp': []}
    
    for k in modes:
        data['mode'].append(k)
        
    for k in reels:
        data['reels'].append(k)
    
    data['spins'].append(len(lstbg))
    data['spins'].append(len(lstfg))
    
    data['probability'].append(len(lstbg) / totalspin)
    data['probability'].append(len(lstfg) / totalspin)    
    
    winnings = 0
    for v in lstbg:
        winnings = winnings + v['realityCheck']['winnings']
        
    data['winnings'].append(winnings)
    data['rtp'].append(winnings / totalspin)
    
    winnings = 0
    for arr in lstfg:
        for v in arr:
            winnings = winnings + v['realityCheck']['winnings']
        
    data['winnings'].append(winnings)
    data['rtp'].append(winnings / totalspin)

    df = pd.DataFrame.from_dict(data)

    return df

def genDataframeWinnings(lstbg, lstfg):
    wins = analyzeWins(lstbg, lstfg)
    
    data = {'winnings':[], 'times': []}
    for k in wins:
        data['winnings'].append(k)
        data['times'].append(wins[k])
        
    df = pd.DataFrame.from_dict(data)
    
    return df

def showWinningsPie(df):
    fig = go.Figure(data=[go.Pie(labels=df['winnings'], values=df['times'])])
    fig.update_layout(title_text='Probability Of Winnings')
    fig.show()
    
def genDataframeSymbols(lstbg, lstfg):
    bg, fg = analyzeReels(lstbg, lstfg, 5)
    
    symbols = {}
    for i in range(5):
        for k in bg[i]:
            if k not in symbols:
                symbols[k] = 1
                
        for k in fg[i]:
            if k not in symbols:
                symbols[k] = 1                
    
    data = {'Symbol': []}
    for k in symbols:
        data['Symbol'].append(k)
    
    for i in range(5):
        data['BASE_REELSET[{}] probability'.format(i)] = []
        
        t = 0
        for k in bg[i]:
            t = t + bg[i][k]
            
        for k in symbols:
            if k in bg[i]:
                data['BASE_REELSET[{}] probability'.format(i)].append(bg[i][k] / t)
            else:
                data['BASE_REELSET[{}] probability'.format(i)].append(0)
                
    for i in range(5):
        data['FREESPIN_REELSET[{}] probability'.format(i)] = []
        
        t = 0
        for k in fg[i]:
            t = t + fg[i][k]
            
        for k in symbols:
            if k in fg[i]:
                data['FREESPIN_REELSET[{}] probability'.format(i)].append(fg[i][k] / t)
            else:
                data['FREESPIN_REELSET[{}] probability'.format(i)].append(0)                
        
    df = pd.DataFrame.from_dict(data)
    
    return df    

def genSymbolWins(lstbg, lstfg):
    bg, fg = analyzeSymbolWins(lstbg, lstfg)
    
    symbols = {}
    for k in bg:
        if k not in symbols:
            symbols[k] = 1

    for k in fg:
        if k not in symbols:
            symbols[k] = 1
            
    nums = {}
    for k in bg:
        for nk in bg[k]:
            if nk not in nums:
                nums[nk] = 1
    
    data = {' Symbol': []}
    
    for k in nums:
        data['BG - X{}'.format(k)] = []
        data['FG - X{}'.format(k)] = []
        
    for k in symbols:
        data[' Symbol'].append(k)
        
        if k in bg:
            for nk in nums:
                if nk in bg[k]:
                    data['BG - X{}'.format(nk)].append(bg[k][nk])
                else:
                    data['BG - X{}'.format(nk)].append(0)
        else:
            for nk in nums:
                data['BG - X{}'.format(nk)].append(0)
                
        if k in fg:
            for nk in nums:
                if nk in fg[k]:
                    data['FG - X{}'.format(nk)].append(fg[k][nk])
                else:
                    data['FG - X{}'.format(nk)].append(0)
        else:
            for nk in nums:
                data['FG - X{}'.format(nk)].append(0)                
    
        
    df = pd.DataFrame.from_dict(data)
    
    return df   

def genMysterySymbolReveal(lstbg, lstfg):
    bg, fg = findMysterySymbolReveal(lstbg, lstfg)
    
    symbols = {}
    bgt = 0
    fgt = 0
    
    for k in bg:
        if k not in symbols and k != '':
            symbols[k] = 1
        
        if k != '':
            bgt = bgt + bg[k]

    for k in fg:
        if k not in symbols and k != '':
            symbols[k] = 1
            
        if k != '':
            fgt = fgt + fg[k]            
    
    
    data = {'MysterySymbolReveal': [], 'BG probability': [], 'FG probability': []}  
    for k in symbols:
        data['MysterySymbolReveal'].append(k)
        
        if k in bg:
            data['BG probability'].append(bg[k] / bgt)
        else:
            data['BG probability'].append(0)            
            
        if k in fg:
            data['FG probability'].append(fg[k] / fgt)            
        else:
            data['FG probability'].append(0)            
    
    df = pd.DataFrame.from_dict(data)
    
    return df   
    

In [3]:
lstbg, lstfg = loadAllFiles()

In [4]:
dfreels = genDataframeModeReels(lstbg, lstfg)

dfreels

,mode,reels,spins,probability,winnings,rtp
0,NORMAL,BASE_REELSET,1343,0.994815,787.40,0.583259
1,FREESPIN,FREESPIN_REELSET,7,0.005185,926.05,0.685963


In [5]:
wins = genDataframeWinnings(lstbg, lstfg)

wins.sort_values(by='winnings')

,winnings,times
0,0.00,979
10,0.20,85
5,0.25,29
14,0.30,19
12,0.40,13
1,0.50,58
32,0.55,4
17,0.60,14
33,0.70,4
2,0.75,11


In [6]:
dfsymbols = genDataframeSymbols(lstbg, lstfg)

dfsymbols.sort_values(by='Symbol')

,Symbol,BASE_REELSET[0] probability,BASE_REELSET[1] probability,BASE_REELSET[2] probability,BASE_REELSET[3] probability,BASE_REELSET[4] probability,FREESPIN_REELSET[0] probability,FREESPIN_REELSET[1] probability,FREESPIN_REELSET[2] probability,FREESPIN_REELSET[3] probability,FREESPIN_REELSET[4] probability
0,L1,0.155407,0.183259,0.029926,0.050074,0.113778,0.200000,0.102857,0.154286,0.157143,0.122857
4,L2,0.164593,0.005333,0.129481,0.155704,0.118222,0.148571,0.137143,0.137143,0.157143,0.162857
1,L3,0.111704,0.162074,0.091259,0.017333,0.147407,0.202857,0.142857,0.137143,0.122857,0.120000
9,L4,0.111704,0.025481,0.107704,0.189926,0.093926,0.102857,0.125714,0.122857,0.122857,0.085714
2,L5,0.088593,0.172889,0.084148,0.081481,0.102667,0.040000,0.025714,0.037143,0.037143,0.017143
5,M1,0.015556,0.015111,0.025037,0.020741,0.034370,0.040000,0.062857,0.085714,0.065714,0.102857
8,M2,0.065333,0.022222,0.165333,0.164889,0.067852,0.048571,0.094286,0.085714,0.091429,0.125714
6,M3,0.049630,0.166519,0.115704,0.027556,0.106222,0.057143,0.117143,0.082857,0.088571,0.097143
7,M4,0.115556,0.033185,0.101185,0.177630,0.076296,0.120000,0.120000,0.105714,0.094286,0.134286
3,M5,0.121926,0.133778,0.056296,0.031704,0.094074,0.040000,0.028571,0.022857,0.022857,0.031429


In [7]:
# showWinningsPie(wins)

In [8]:
dfsymbolwin = genSymbolWins(lstbg, lstfg)

lst = list(dfsymbolwin.columns)
lst.sort()
dfsymbolwin[lst].sort_values(by=' Symbol')

,Symbol,BG - X3,BG - X4,BG - X5,FG - X3,FG - X4,FG - X5
0,L1,127,58,0,1,168,0
1,L2,31,16,17,32,0,46
7,L3,112,43,21,9,50,107
8,L4,18,34,8,10,55,68
5,L5,81,41,26,0,0,0
9,M1,0,0,0,0,12,0
2,M2,12,6,4,36,0,0
3,M3,87,17,40,0,9,45
6,M4,35,26,99,0,50,126
4,M5,72,34,10,0,4,0


In [9]:
dfmystery = genMysterySymbolReveal(lstbg, lstfg)

dfmystery.sort_values(by='MysterySymbolReveal')

,MysterySymbolReveal,BG probability,FG probability
2,L1,0.127193,0.191176
1,L2,0.164474,0.147059
3,L3,0.122807,0.176471
0,L4,0.149123,0.220588
7,L5,0.127193,0.029412
9,M1,0.013158,0.044118
5,M2,0.032895,0.029412
4,M3,0.072368,0.044118
6,M4,0.074561,0.102941
8,M5,0.116228,0.014706


In [10]:
# lstm = findMystery(lstbg)

# lstm[0]